In [21]:
import cv2
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import pandas as pd

In [66]:
# video paths
CellA_GFP = 'data/CellA_GFP.avi'
CellA_noDox = 'data/CellA_noDoxycycline.avi'
CellA_ShME480 = 'data/CellA_ShME480.avi'
CellB_GFP = 'data/CellB_GFP.avi'
CellB_ME480 = 'data/CellB_ME480.avi'

In [63]:
def process_video(video_path, display_video=False, display_velocity=50):
    cap = cv2.VideoCapture(video_path)
    
    # Get properties of the video
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    ret = True
    frame_count = 1
    # create an empty dataframe to store the results
    table = pd.DataFrame(columns=['area', 'perimeter', 'major_axis', 'minor_axis', 'angle', 'circularity', 'aspect_ratio', 'roundness', 'solidity'])

    while ret:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray_frame, (5,5),0)
    
        # Apply thresholding
        h, mask = cv2.threshold(blur, 128, 200, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        
        # apply erosion to fill in the holes
        kernel = np.ones((11, 11),np.uint8)
        mask = cv2.erode(mask,kernel,iterations = 2)
        mask = cv2.dilate(mask,kernel,iterations = 2)

        contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # we select only frames with 2 contours (1 for the frame frame_countour, and 1 for the cell contour)
        if len(contours) == 2:
            gray_frame = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)
            # sort the contours by size to have the frame contour first, cell contour will always be smaller
            cnt = sorted(contours, key=cv2.contourArea, reverse=True) 
            
            # select the cell contour by size to get rid of noisy contours
            if cv2.contourArea(cnt[1]) > 1200 and cv2.contourArea(cnt[1]) < 3000:
                
                # contourIdx allows to choose the contour to draw
                cv2.drawContours(gray_frame, cnt, contourIdx=1, color=(0,255,0), thickness=1)

                # fit an ellipse to the contour and draw it
                ellipse = cv2.fitEllipse(cnt[1])
                (xc, yc), (d1, d2), angle = ellipse
                
                # add the ellipse parameters to the dataframe
                # columns=['area', 'perimeter', 'major_axis', 'minor_axis', 'angle', 'circularity', 'aspect_ratio', 'roundness', 'solidity']
                table.loc[frame_count] = [cv2.contourArea(cnt[1]), cv2.arcLength(cnt[1], True), max(d1,d2), min(d1,d2), angle, 4*np.pi*cv2.contourArea(cnt[1])/(cv2.arcLength(cnt[1], True)**2), max(d1,d2)/min(d1,d2), 4*cv2.contourArea(cnt[1])/(np.pi*max(d1,d2)**2), cv2.contourArea(cnt[1])/cv2.contourArea(cv2.convexHull(cnt[1]))]
                
                if display_video:
                    # draw the ellipse on the gray frame
                    cv2.ellipse(gray_frame, ellipse, (0,0,255), 1)

                    # transform mask to 3 channels to concatenate it with the frame
                    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

                    # create a black image of 256x10 pixels for display separation
                    black = np.zeros((frame_height, 10, 3), np.uint8)

                    # add a white frame of 256x128 pixels to display the text
                    text_screen = np.ones((frame_height, 160, 3), np.uint8)*255

                    # Display the resulting frames mask and gray_frame in the same window 
                    display = np.concatenate((text_screen, black, frame, black, mask, black, gray_frame), axis=1)

                    # add a text to the image
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(display, 'frame {}'.format(frame_count), (10, 20), font, 0.5, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(display, 'area: {:.2f}'.format(cv2.contourArea(cnt[1])), (10, 40), font, 0.5, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(display, 'perimeter: {:.2f}'.format(cv2.arcLength(cnt[1], True)), (10, 60), font, 0.5, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(display, 'major_axis: {:.2f}'.format(max(d1,d2)), (10, 80), font, 0.5, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(display, 'minor_axis: {:.2f}'.format(min(d1,d2)), (10, 100), font, 0.5, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(display, 'angle: {:.2f}'.format(angle), (10, 120), font, 0.5, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(display, 'circularity: {:.2f}'.format(4*np.pi*cv2.contourArea(cnt[1])/(cv2.arcLength(cnt[1], True)**2)), (10, 140), font, 0.5, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(display, 'aspect_ratio: {:.2f}'.format(max(d1,d2)/min(d1,d2)), (10, 160), font, 0.5, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(display, 'roundness: {:.2f}'.format(4*cv2.contourArea(cnt[1])/(np.pi*max(d1,d2)**2)), (10, 180), font, 0.5, (0,0,0), 1, cv2.LINE_AA)
                    cv2.putText(display, 'solidity: {:.2f}'.format(cv2.contourArea(cnt[1])/cv2.contourArea(cv2.convexHull(cnt[1]))), (10, 200), font, 0.5, (0,0,0), 1, cv2.LINE_AA)

                    
                    cv2.imshow('frame', display)

                frame_count += 1

        if cv2.waitKey(display_velocity) & 0xFF == ord('q'):
            break

        
    cv2.destroyAllWindows()
    cap.release()
    return table


In [71]:
data_CellA_GFP = process_video(CellA_GFP, display_video=True, display_velocity=100)

In [72]:
data_CellA_GFP

,area,perimeter,major_axis,minor_axis,angle,circularity,aspect_ratio,roundness,solidity
1,1780.5,165.296463,60.056301,38.369957,0.217590,0.818888,1.565191,0.628543,0.978835
2,1427.5,156.124891,58.972778,31.673285,0.960593,0.735939,1.861909,0.522616,0.969440
3,2636.0,196.852812,69.329933,49.037415,0.062943,0.854815,1.413817,0.698255,0.985973
4,2586.0,196.852812,69.698555,48.014324,0.491395,0.838601,1.451620,0.677784,0.979175
5,2044.0,181.195958,68.280678,38.725574,0.253152,0.782337,1.763193,0.558207,0.972176
...,...,...,...,...,...,...,...,...,...
194,1817.5,177.296463,66.382179,35.039204,2.533469,0.726581,1.894512,0.525148,0.955071
195,1832.5,173.781745,67.690750,34.641758,0.474903,0.762510,1.954022,0.509208,0.979423
196,2115.0,191.539104,72.317406,37.526405,0.354869,0.724445,1.927107,0.514914,0.973757
197,1980.0,182.710677,68.484406,38.359867,6.513602,0.745328,1.785314,0.537517,0.954677


In [73]:
data_CellA_noDox = process_video(CellA_noDox, display_video=True, display_velocity=10) # TUNE PARAMETERS

In [68]:
data_CellA_ShME480 = process_video(CellA_ShME480, display_video=True, display_velocity=10)

In [69]:
data_CellB_GFP = process_video(CellB_GFP, display_video=True, display_velocity=10)

In [70]:
data_CellB_ME480 = process_video(CellB_ME480, display_video=True, display_velocity=10) # TUNE PARAMETERS